In [6]:
import time
from browsermobproxy import Server
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import json

server = Server('/anaconda3/bin/browsermob-proxy-2.1.4/bin/browsermob-proxy')
server.start()
proxy = server.create_proxy()

options = webdriver.ChromeOptions()
options.add_argument(
    '--proxy-server={host}:{port}'.format(host='localhost', port=proxy.port))
browser = webdriver.Chrome('/usr/local/bin/chromedriver',
                           options=options)

browser.get('https://mrbostondrinks.com/')
time.sleep(1)

browser.execute_script(
    'window.localStorage.setItem("ngStorage-verified", "true")')
time.sleep(1)

proxy.new_har('req',options={'captureHeaders': True,'captureContent':True})
time.sleep(1)

ed_1935 = 'old-mr-boston-deluxe-official-bartenders-guide-1935'
ed_1957 = 'old-mr-boston-de-luxe-official-bartenders-guide-1957'
ed_1960 = 'old-mr-boston-de-luxe-official-bartenders-guide-1960'
ed_1964 = 'old-mr-boston-de-luxe-official-bartenders-guide-1964'
ed_1972 = 'old-mr-boston-de-luxe-official-bartenders-guide-1972'
ed_1974 = 'mr-boston-deluxe-official-bartenders-guide-1974'
ed_1978 = 'mr-boston-deluxe-official-bartenders-guide-1978'
ed_1984 = 'mr-boston-official-bartenders-guide-50th-anniversary-edition-1984'
ed_1988 = 'mr-boston-official-bartenders-guide-1988'
ed_2000 = 'mr-boston-official-bartenders-and-party-guide-2000-1'
ed_2012 = 'mr-boston-official-bartenders-guide-2012'
editions = [ed_1935, ed_1957, ed_1960, ed_1964, ed_1972, ed_1974, ed_1978,
            ed_1984, ed_1988, ed_2000, ed_2012]

master_recipes = {}
for edition in editions:
    browser.get('https://mrbostondrinks.com/books/' + edition)
    time.sleep(1)
    
    see_more_button = browser.find_elements_by_xpath(
        '//button[@id="see-more"]')
    while len(see_more_button) > 0:
        try:
            see_more_button[0].click()
            time.sleep(1)
        except StaleElementReferenceException:
            break
        
    ed_recipes = []
    for dictionary in proxy.har['log']['entries']:
        if dictionary['request']['url'].startswith(
            'https://api.mrboston.recipes/book/' + edition):
                ed_recipes.append(json.loads(
                    dictionary['response']['content']['text'])
                                  ['recipes']['data'])
    ed_recipes = [recipe for sublist in ed_recipes for recipe in sublist]
    
    master_recipes[edition] = ed_recipes
    print(len(master_recipes[edition]), '\t', edition)

In [52]:
with open('master_recipes.json', 'w') as json_file:
    json.dump(master_recipes, json_file)

In [1]:
with open('master_recipes.json') as json_file:
    master_recipes = json.load(json_file)

for edition in master_recipes:
    print(len(master_recipes[edition]), '\t', edition)

673 	 old-mr-boston-deluxe-official-bartenders-guide-1935
758 	 old-mr-boston-de-luxe-official-bartenders-guide-1957
737 	 old-mr-boston-de-luxe-official-bartenders-guide-1960
757 	 old-mr-boston-de-luxe-official-bartenders-guide-1964
735 	 old-mr-boston-de-luxe-official-bartenders-guide-1972
1004 	 mr-boston-deluxe-official-bartenders-guide-1974
999 	 mr-boston-deluxe-official-bartenders-guide-1978
1075 	 mr-boston-official-bartenders-guide-50th-anniversary-edition-1984
1101 	 mr-boston-official-bartenders-guide-1988
1174 	 mr-boston-official-bartenders-and-party-guide-2000-1
1523 	 mr-boston-official-bartenders-guide-2012
